In [1]:
import folium
import folium.plugins as plugins
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import geopandas

covid_dataframe = pd.read_csv('spread_hist_by_state.csv')
covid_dataframe.columns

Index(['date', 'state', 'positive', 'negative', 'pending',
       'hospitalizedCurrently', 'hospitalizedCumulative', 'inIcuCurrently',
       'inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative',
       'recovered', 'dataQualityGrade', 'lastUpdateEt', 'hash', 'dateChecked',
       'death', 'hospitalized', 'total', 'totalTestResults', 'posNeg', 'fips',
       'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease',
       'positiveIncrease', 'totalTestResultsIncrease'],
      dtype='object')

In [2]:
covid_data = covid_dataframe[['date', 'state', 'positive','total','totalTestResults','dataQualityGrade', 'lastUpdateEt','recovered']].loc[covid_dataframe['dataQualityGrade'] != 'F']
covid_data.head(3)

,date,state,positive,total,totalTestResults,dataQualityGrade,lastUpdateEt,recovered
0,20200513,AK,383.0,30649,30649,B,5/13/2020 0:00,338.0
1,20200513,AL,10617.0,136372,136372,B,5/13/2020 0:00,NaN
2,20200513,AR,4164.0,73215,73215,A,5/12/2020 18:33,3220.0


In [3]:
state_geo_df = pd.read_csv('state_latlong.csv')
covid_df = pd.merge(covid_data, state_geo_df, how='left', left_on="state", right_on='state')
#covid_df

In [27]:
plot_df = covid_df[['latitude','longitude','date','total']].sort_values(by='date', axis=0, ascending=True)
plot_df.head(30|90)

,latitude,longitude,date,total
3913,47.751074,-120.740139,20200122,1
3912,47.751074,-120.740139,20200123,1
3911,47.751074,-120.740139,20200124,1
3910,47.751074,-120.740139,20200125,1
3909,47.751074,-120.740139,20200126,1
...,...,...,...,...
3805,39.045755,-76.641271,20200306,41
3804,37.839333,-84.270018,20200306,10
3803,39.011902,-98.484246,20200306,4
3801,40.633125,-89.398528,20200306,220


In [5]:
#gdf = geopandas.GeoDataFrame(
#    plot_df, geometry=geopandas.points_from_xy(plot_df.longitude, plot_df.latitude))
#gdf

In [14]:
#  zoom_start=6
params = {
    'location':[plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    'tiles':'stamentoner',
    'min_lat':plot_df['latitude'].min(),
    'max_lat':plot_df['latitude'].max(),
    'min_lon':plot_df['longitude'].min(),
    'max_lon':plot_df['longitude'].max()
}
#        min_lat=plot_df['latitude'].min(),
#        max_lat=plot_df['latitude'].max(),
#        min_lon=plot_df['longitude'].min(),
#        max_lon=plot_df['longitude'].max(),
covid_map = folium.Map(
        [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
        tiles='stamentoner',
        zoom_start=5
    )


#heat_df = plot_df.dropna(axis=0, subset=['latitude','longitude'])
#heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

heat_df = plot_df.loc[plot_df['date'] == 20200315].dropna(axis=0, subset=['latitude','longitude', 'total', 'date'])
print(f'heat_df {len(heat_df)}')
#heat_df['weight'] = heat_df['total'].astype(float)
#heat_data = [[[row['latitude'],row['longitude']] for index, row in heat_df[heat_df['weight'] == i].iterrows()] for i in range(0,plot_df['total'].max())]

#heat_time = list(heat_df.dropna(axis=0, subset=['date']))
#heat_time =  [[row['date']] for index, row in heat_df.iterrows()]
#heat_time =  [[index] for index, row in heat_df.iterrows()]

#heat_time = plot_df[])#.toList()
heat_data = []
for row in heat_df.itertuples():
    #print(row.total)
    heat_data.append([row.latitude,row.longitude, row.total])# for i in range(0, row.total)]
#    [heat_data.append([row.latitude,row.longitude, row.total]) for i in range(0, row.total)]
#heat_data = [[row['latitude'],row['longitude'],row['total']] for index, row in heat_df.iterrows()]
len(heat_data)

                
#     for index, row in heat_df.iterrows()]
print(heat_data[:3])

HeatMap(heat_data).add_to(covid_map)
covid_map

heat_df 51
[[45.253783, -69.44546899999999, 111], [41.878003, -93.097702, 101], [39.045755, -76.641271, 125]]


In [31]:
#help(plugins.HeatMapWithTime)
#help(folium.Icon)

In [ ]:
date_data = []
date_indx = []

for date_stp in plot_df['date'].unique():
    heat_df = plot_df.loc[plot_df['date'] == date_stp].dropna(axis=0, subset=['latitude','longitude', 'total','date'])
    dt = str(date_stp)
    y = int(dt[:4])
    m = int(dt[4:6])
    d = int(dt[6:])
    date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))

    heat_data = []
    for row in heat_df.itertuples():
        #print(row.total)
        heat_data.append([row.latitude,row.longitude, row.total])
    print(f'heat_data {row.date}\n{"--"* 40}\n{heat_data}\n')
    date_data.append(heat_data)
        #print(f'y {y}|m {m}|d {d}')
#print(f'{datetime(y,m,d).strftime("%Y-%m-%d")}')
print(f'date_data [{len(date_data)}] date_indx [{len(date_indx)}]')
print(f'date_data {"--"* 40}\n{date_data}')

covid_heat_time_map = folium.Map(
    [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    tiles='stamentoner',
    zoom_start=4,
    max_opacity=0.6
) 

HeatMapwtime = plugins.HeatMapWithTime(
    data=date_data,
    index=date_indx,
    auto_play=True
)

HeatMapwtime.add_to(covid_heat_time_map)
covid_heat_time_map

In [ ]:
heat_map = plugins.HeatMap(heat_data,max_opacity=0.8)
# Plot it on the map
heat_map.add_to(covid_map)
covid_map

In [ ]:
# extract lat, lon, and magnitude for the folium heatmap
date_data = []
date_indx = []

heat_df = plot_df.dropna(axis=0, subset=['latitude','longitude', 'total','date'])
    
for date_stp in plot_df['date'].unique():
    
lats = [float(item[2]) for item in plot_df]
lons = [float(item[3]) for item in plot_df]
mag = [float(item[4]) for item in plot_df]

    dt = str(date_stp)
    y = int(dt[:4])
    m = int(dt[4:6])
    d = int(dt[6:])
    date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))


In [ ]:

industriesDF = pd.read_csv('merged_industries.csv')
industriesDF